In [ ]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import math
import matplotlib.pyplot as plt
import os
import numpy as np
EPOCHS = 50

FOLDER = "./Data/"

MODEL_DIR = FOLDER +"Vae_model_dict_50EPOCHS_v2"
DIGIT_SAVE_DIR = FOLDER + 'sample_digits.png'
z_dim = 4
RANDOM_VECTOR = FOLDER + "random_vector.txt"

TRAIN_LOSS = FOLDER + "TRAIN_LOSS_VAE.txt.npy"
TEST_LOSS = FOLDER + "TEST_LOSS_VAE.txt.npy"
TRAIN_KLD = FOLDER + "TRAIN_KLD_VAE.txt.npy"
TEST_KLD = FOLDER + "TEST_KLD_VAE.txt.npy"

LOSS_PLOT = FOLDER + "LOSS_PLOT.png"
KLD_PLOT = FOLDER + "KLD_PLOT.png"
if not os.path.exists(RANDOM_VECTOR):
    z = torch.randn(100, 4)
    torch.save(z, RANDOM_VECTOR)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [ ]:
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, std):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + std - mu.pow(2) - std.exp())
    return BCE + KLD, KLD

In [ ]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim, loss_function):
        super(VAE, self).__init__()
        self.loss_function = loss_function
        self.zdim = z_dim
        # encoder part
        self.lstm1 = nn.LSTM(input_size=x_dim, hidden_size=h_dim1)
        self.fc31 = nn.Linear(h_dim1, z_dim)
        self.fc32 = nn.Linear(h_dim1, z_dim)
        # decoder part
        self.tconv2d1 = nn.ConvTranspose2d(1, 32, 8)
        self.tconv2d2 = nn.ConvTranspose2d(32, 16, 4)
        self.tconv2d3 = nn.ConvTranspose2d(16, 8, 2)
        self.fc2 = nn.Linear(h_dim2, x_dim)
        
    def encoder(self, x):
        h, hidden = self.lstm1(x)
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = torch.reshape(z, (-1, 1, int(math.sqrt(self.zdim)), int(math.sqrt(self.zdim)),))
        h = F.relu(self.tconv2d1(h))
        #h = torch.nn.functional.dropout(h, p=0.8)
        h = F.relu(self.tconv2d2(h))
        #h = torch.nn.functional.dropout(h, p=0.8)
        h = F.relu(self.tconv2d3(h))
        #h = torch.nn.functional.dropout(h, p=0.8)
        h = torch.flatten(h, start_dim=1)
        return F.sigmoid(self.fc2(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var
    
    def train_data(self, epoch, train_loader):
        self.train()
        train_loss = 0
        train_kld = 0
        loss_history = []
        KLD_history = []
        for batch_idx, (data, _) in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()
            
            recon_batch, mu, log_var = vae(data)
            loss, KLD= self.loss_function(recon_batch, data, mu, log_var)
            loss.backward()
            train_loss += loss.item()
            train_kld += KLD.item()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item() / len(data)))
        
        print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))
        return train_loss / len(train_loader.dataset), train_kld / len(train_loader.dataset)
    
    def test_data(self, test_loader):
        self.eval()
        test_loss= 0
        kld_loss = 0
        with torch.no_grad():
            for data, _ in test_loader:
                data = data.to(device)
                recon, mu, log_var = vae(data)
                
                # sum up batch loss
                loss, kld = self.loss_function(recon, data, mu, log_var)
                test_loss += loss.item()
                kld_loss += kld.item()
        test_loss /= len(test_loader.dataset)
        kld_loss /= len(test_loader.dataset)
        print('====> Test set loss: {:.4f}'.format(test_loss))
        return test_loss, kld_loss

# build model

vae = VAE(x_dim=784, h_dim1= 128, h_dim2=1352, z_dim=z_dim, loss_function=loss_function)
vae = vae.to(device)
optimizer = optim.Adam(vae.parameters())

In [ ]:
train_loss_history = []
train_kld_history = []
test_loss_history = []
test_kld_history = []
for epoch in range(EPOCHS):
    l,k = vae.train_data(epoch, train_loader)
    train_loss_history.append(l)
    train_kld_history.append(k)
    l,k = vae.test_data(test_loader)
    test_loss_history.append(l)
    test_kld_history.append(k)

In [ ]:
np.save(TRAIN_LOSS, np.array(train_loss_history))
np.save(TEST_LOSS, np.array(test_loss_history))
np.save(TRAIN_KLD, np.array(train_kld_history))
np.save(TEST_KLD, np.array(test_kld_history))

In [ ]:
torch.save(vae.state_dict(), MODEL_DIR)

In [ ]:
train_loss_history = np.load(TRAIN_LOSS)
test_loss_history = np.load(TEST_LOSS)
train_kld_history = np.load(TRAIN_KLD)
test_kld_history = np.load(TEST_KLD)

In [ ]:
plt.figure(1)
plt.plot(range(1,EPOCHS+1), train_loss_history)
plt.plot(range(1,EPOCHS+1), test_loss_history)
plt.xlabel("EPOCHS")
plt.ylabel("Average loss")
plt.legend(["Train", "Test"])
plt.savefig(LOSS_PLOT)

In [ ]:
plt.figure(2)
plt.plot(range(1,EPOCHS+1), train_kld_history)
plt.plot(range(1,EPOCHS+1), test_kld_history)
plt.xlabel("EPOCHS")
plt.ylabel("Average kld")
plt.legend(["Train", "Test"])
plt.savefig(KLD_PLOT)

In [ ]:
vae = VAE(x_dim=784, h_dim1= 128, h_dim2=1352, z_dim=z_dim, loss_function=loss_function)
vae = vae.to(device)
vae.load_state_dict(torch.load(MODEL_DIR, map_location=device))

with torch.no_grad():
    z = torch.load(RANDOM_VECTOR).to(device)
    sample = vae.decoder(z).to(device)
    save_image(sample.view(100, 1, 28, 28), DIGIT_SAVE_DIR)